**Keras MNIST PQT/QAT Tutorial**

Load tensorflow library

In [ ]:
import os 
import tensorflow as tf
print(tf.__version__)

!pip install tensorflow_model_optimization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/Colab_Embedded2023'

Load dataset (MNIST)

In [ ]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = y_train.flatten()
y_test = y_test.flatten()


Preprocessing dataset

In [ ]:
import numpy as np
input_shape = (32, 32, 3)

x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_train = x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], *input_shape)
x_test = x_test / 255.0

y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)


Define and train the baseline model 

In [ ]:
batch_size = 64
epochs = 50


reg = tf.keras.regularizers.l2(1e-4)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', input_shape=input_shape, kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(16, 3, padding='same', kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),    

    tf.keras.layers.Conv2D(32, 3, padding='same', kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(32, 3, padding='same', kernel_regularizer=reg),    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, 3, padding='same', kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(64, 3, padding='same', kernel_regularizer=reg),    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax', kernel_regularizer=reg)
])

schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-3, decay_steps=x_train.shape[0] * epochs // batch_size)


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=schedule, weight_decay=0),
            loss='categorical_crossentropy', metrics=['acc'])


from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )

#history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch = len(x_train) / batch_size, epochs=epochs, validation_data=(x_test, y_test))

In [ ]:
tf.keras.models.save_model(model, os.path.join(base_path, "model.h5"))
model.save_weights(os.path.join(base_path, "model.weight"))

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
tflite_model = converter.convert()
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Save the model.
with open(os.path.join(base_path, 'model.tflite'), 'wb') as f:
  f.write(tflite_model)

Apply Quantization-aware Training

In [ ]:
from tensorflow_model_optimization.quantization.keras import quantize_apply
from tensorflow_model_optimization.quantization.keras import quantize_annotate_layer

batch_size = 64
epochs = 5

reg = tf.keras.regularizers.l2(1e-4)

model = tf.keras.models.Sequential([
    quantize_annotate_layer(tf.keras.layers.Conv2D(16, 3, padding='same', input_shape=input_shape, kernel_regularizer=reg)),
    quantize_annotate_layer(tf.keras.layers.BatchNormalization()),
    quantize_annotate_layer(tf.keras.layers.ReLU()),
    quantize_annotate_layer(tf.keras.layers.Conv2D(16, 3, padding='same', kernel_regularizer=reg)),
    quantize_annotate_layer(tf.keras.layers.BatchNormalization()),
    quantize_annotate_layer(tf.keras.layers.ReLU()),
    quantize_annotate_layer(tf.keras.layers.MaxPooling2D()),    

    quantize_annotate_layer(tf.keras.layers.Conv2D(32, 3, padding='same', kernel_regularizer=reg)),
    quantize_annotate_layer(tf.keras.layers.BatchNormalization()),
    quantize_annotate_layer(tf.keras.layers.ReLU()),
    quantize_annotate_layer(tf.keras.layers.Conv2D(32, 3, padding='same', kernel_regularizer=reg)),    
    quantize_annotate_layer(tf.keras.layers.BatchNormalization()),
    quantize_annotate_layer(tf.keras.layers.ReLU()),
    quantize_annotate_layer(tf.keras.layers.MaxPooling2D()),

    quantize_annotate_layer(tf.keras.layers.Conv2D(64, 3, padding='same', kernel_regularizer=reg)),
    quantize_annotate_layer(tf.keras.layers.BatchNormalization()),
    quantize_annotate_layer(tf.keras.layers.ReLU()),
    quantize_annotate_layer(tf.keras.layers.Conv2D(64, 3, padding='same', kernel_regularizer=reg)),    
    quantize_annotate_layer(tf.keras.layers.BatchNormalization()),
    quantize_annotate_layer(tf.keras.layers.ReLU()),
    
    quantize_annotate_layer(tf.keras.layers.GlobalAveragePooling2D()),
    quantize_annotate_layer(tf.keras.layers.Flatten()),
    quantize_annotate_layer(tf.keras.layers.Dense(10, activation='softmax', kernel_regularizer=reg))
])

model.load_weights(os.path.join(base_path, "model.weight"))

schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-3, decay_steps=x_train.shape[0] * epochs // batch_size)



from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )



quantized_model = quantize_apply(model)
quantized_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=schedule, weight_decay=0),
            loss='categorical_crossentropy', metrics=['acc'])

quantized_model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch = len(x_train) / batch_size, epochs=epochs, validation_data=(x_test, y_test))


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model) # path to the SavedModel directory

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model.
with open(os.path.join(base_path, 'model_quant.tflite'), 'wb') as f:
  f.write(tflite_model)

In [ ]:
print("Float model in Kb:", os.path.getsize(os.path.join(base_path, "model.tflite")) / float(2**10))
print("Quantized model in Kb:",  os.path.getsize(os.path.join(base_path, "model_quant.tflite")) / float(2**10))
